In [ ]:
!pip install torch_geometric

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

In [3]:
np.random.seed(0)
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
transform=T.Compose(
    [
        T.NormalizeFeatures(),
        T.ToDevice(device),
        T.RandomLinkSplit(num_val=.05, num_test=.1, is_undirected=True, split_labels=True, add_negative_train_samples=False)
    ]
)

In [5]:
data = Planetoid(root='.', name='Cora', transform=transform)

Processing...
Done!


In [6]:
train_data, val_data, test_data = data[0]

In [7]:
train_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[4488], pos_edge_label_index=[2, 4488])

In [8]:
val_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[263], pos_edge_label_index=[2, 263], neg_edge_label=[263], neg_edge_label_index=[2, 263])

In [9]:
test_data

Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[527], pos_edge_label_index=[2, 527], neg_edge_label=[527], neg_edge_label_index=[2, 527])

In [10]:
from torch_geometric.nn import GCNConv, VGAE

In [11]:
class Encoder(torch.nn.Module):
  def __init__(self, dim_in, dim_out):
    super().__init__()
    self.conv1 = GCNConv(dim_in, 2*dim_out)
    self.conv_mu = GCNConv(2*dim_out, dim_out)
    self.conv_logstd = GCNConv(2*dim_out, dim_out)
  def forward(self, X, edge_index):
    h = self.conv1(X, edge_index)
    return self.conv_mu(h, edge_index), self.conv_logstd(h, edge_index)

In [12]:
model = VGAE(Encoder(data.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=.01)

In [24]:
def train():
  model.train()
  optimizer.zero_grad()
  Z = model.encode(train_data.x, train_data.edge_index)
  l_elbo = model.recon_loss(z=Z, pos_edge_index=train_data.pos_edge_label_index) + (1/train_data.num_nodes)*model.kl_loss()
  l_elbo.backward()
  optimizer.step()
  return float(l_elbo.detach())

In [25]:
@torch.no_grad()
def test():
  model.eval()
  Z = model.encode(test_data.x, test_data.edge_index)
  acc = model.test(Z, pos_edge_index=test_data.pos_edge_label_index, neg_edge_index=test_data.neg_edge_label_index)
  return acc

In [26]:
epochs = 301
for epoch in range(epochs):
  x=train()
  if epoch % 100 ==0:
    print(f"Epoch:{epoch}| Loss:{x}")
acc = test()
print(f"AUC:{acc[0]}|AP:{acc[1]}")

Epoch:0| Loss:0.9004015326499939
Epoch:100| Loss:0.8862262964248657
Epoch:200| Loss:0.858666181564331
Epoch:300| Loss:0.8525027632713318
AUC:0.9157380035934308|AP:0.921115443762428


In [28]:
 Z = model.encode(test_data.x, test_data.edge_index)
 A_hat = torch.sigmoid(Z @ Z.T)
 A_hat

tensor([[0.8890, 0.3946, 0.6908,  ..., 0.4717, 0.8234, 0.7794],
        [0.3946, 0.7847, 0.5278,  ..., 0.4629, 0.5064, 0.4638],
        [0.6908, 0.5278, 0.7736,  ..., 0.3594, 0.8267, 0.7580],
        ...,
        [0.4717, 0.4629, 0.3594,  ..., 0.9062, 0.4310, 0.4612],
        [0.8234, 0.5064, 0.8267,  ..., 0.4310, 0.9464, 0.9134],
        [0.7794, 0.4638, 0.7580,  ..., 0.4612, 0.9134, 0.8815]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)